## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,Georgetown,5.4112,100.3354,85.93,81,20,1.01,MY,2022-01-30 02:53:26,haze
1,1,Puerto Ayora,-0.7393,-90.3518,73.42,85,74,5.93,EC,2022-01-30 02:56:54,broken clouds
2,2,Bethel,41.3712,-73.4140,15.57,55,100,5.01,US,2022-01-30 03:02:18,overcast clouds
3,3,Iqaluit,63.7506,-68.5145,-13.27,62,40,8.99,CA,2022-01-30 03:00:15,scattered clouds
4,4,Butaritari,3.0707,172.7902,82.49,72,52,17.83,KI,2022-01-30 03:05:51,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp_criteria = float(input("What is the maximum temperature you would like for your trip? "))
min_temp_criteria = float(input("What is the minimum temperature you would like for your trip? "))

What is the maximum temperature you would like for your trip? 75
What is the minimum temperature you would like for your trip? 55


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temp_criteria = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp_criteria) & 
                                 (city_data_df["Max Temp"] >= min_temp_criteria)]

In [7]:
# 4a. Determine if there are any empty rows.
temp_criteria.count()

City_ID        175
City           175
Lat            175
Lng            175
Max Temp       175
Humidity       175
Cloudiness     175
Wind Speed     175
Country        172
Date           175
Description    175
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_temp_criteria = temp_criteria.dropna()

In [9]:
new_temp_criteria.count()

City_ID        172
City           172
Lat            172
Lng            172
Max Temp       172
Humidity       172
Cloudiness     172
Wind Speed     172
Country        172
Date           172
Description    172
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_temp_criteria[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,73.42,broken clouds,-0.7393,-90.3518,
6,Marsh Harbour,BS,59.86,broken clouds,26.5412,-77.0636,
11,Victoria,HK,62.04,broken clouds,22.2855,114.1577,
12,Tocopilla,CL,66.61,few clouds,-22.0920,-70.1979,
20,Bluff,NZ,59.99,overcast clouds,-46.6000,168.3333,
22,Qandala,SO,71.80,overcast clouds,11.4720,49.8728,
26,Tuatapere,NZ,67.73,overcast clouds,-46.1333,167.6833,
29,Nishihara,JP,68.05,broken clouds,26.1842,127.7558,
36,Cabo San Lucas,MX,72.52,clear sky,22.8909,-109.9124,
37,Chadiza,ZM,64.85,light rain,-14.0678,32.4392,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found...skipping.')
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [27]:
hotel_df.count()

City           172
Country        172
Max Temp       172
Description    172
Lat            172
Lng            172
Hotel Name     172
dtype: int64

In [9]:
# 7. Drop the rows where there is no Hotel Name.
# n/a per above hotel_df.count()

In [29]:
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,73.42,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,Marsh Harbour,BS,59.86,broken clouds,26.5412,-77.0636,Abaco Beach Resort
11,Victoria,HK,62.04,broken clouds,22.2855,114.1577,Mini Hotel Central
12,Tocopilla,CL,66.61,few clouds,-22.0920,-70.1979,Hotel Galvarino
20,Bluff,NZ,59.99,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park


In [30]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))